# Getting Started with `deph`

`deph` is a powerful tool for analyzing and isolating Python code dependencies. It's particularly useful for creating minimal, reproducible code snippets from complex projects or interactive notebook sessions.

This notebook will walk you through the two main functions of `deph`:

1.  **`deph.analyze()`**: To inspect the dependencies of a function or class.
2.  **`deph.isolate()`**: To generate a self-contained, runnable script from a function or class.

## 1. Setup

First, let's make sure `deph` is installed and import necessary libraries. If you are running this from the repository root, the library should be discoverable.

In [7]:
import sys
import numpy as np
import textwrap
from pathlib import Path
from pprint import pprint
module_dir = Path.cwd().parent / 'src'
sys.path.append(module_dir.as_posix())
# Import the main functions from deph
try:
    from deph import analyze, isolate
except ImportError:
    print("Deph not found. Please install it or run this notebook from the project root.")

## 2. Defining Example Functions

Let's create a few functions with different kinds of dependencies to use as our test cases.

- A function that calls another local function.
- A function that uses an external library (`numpy`).
- A function that uses a standard library (`textwrap`).

In [8]:
def get_greeting(name: str) -> str:
    """A simple helper function."""
    return f"Hello, {name}!"

def process_data(data, name):
    """
    This function uses:
    1. An external library (numpy).
    2. A standard library (textwrap).
    3. A local function (get_greeting).
    """
    greeting = get_greeting(name)
    print(greeting)
    
    # Use numpy for some calculation
    mean_value = np.mean(data)
    print(f"Mean of data: {mean_value}")
    
    # Use textwrap to format a message
    long_message = f"The calculated mean was {mean_value}. This is a long message that needs to be wrapped nicely for display."
    wrapped_message = textwrap.fill(long_message, width=40)
    
    print("\n--- Formatted Message ---")
    print(wrapped_message)
    
    return mean_value

## 3. Analyzing Dependencies with `deph.analyze()`

The `analyze` function inspects an object and returns a detailed report of its dependencies. This includes imported modules, module-level variables, and other functions or classes it depends on.

In [9]:
# Let's analyze our main function, `process_data`
report = analyze(process_data)

# The report is a dictionary-like object. Let's inspect its contents.
pprint(report)

AttrDefaultDict(
  {'def_items': [DefItem(name='process_data',
                         type='function',
                         code='def process_data(data, name):\n'
                              '    """\n'
                              '    This function uses:\n'
                              '    1. An external library (numpy).\n'
                              '    2. A standard library (textwrap).\n'
                              '    3. A local function (get_greeting).\n'
                              '    """\n'
                              '    greeting = get_greeting(name)\n'
                              '    print(greeting)\n'
                              '    mean_value = np.mean(data)\n'
                              "    print(f'Mean of data: {mean_value}')\n"
                              "    long_message = f'The calculated mean was "
                              '{mean_value}. This is a long message that needs to be '
                              "wrapped nicely 

### Understanding the Report

The report contains several key sections:

- **`entries`**: The list of objects you asked to analyze.
- **`def_items`**: A list of all function and class definitions required by the entries. You can see that `deph` automatically found that `process_data` needs `get_greeting`.
- **`imports`**: A dictionary of all required imports, categorized by module. Notice it found `numpy` and `textwrap`.
- **`unbound`**: A list of any names that were used but could not be resolved. This should be empty if all dependencies are found.

## 4. Isolating Code with `deph.isolate()`

The `isolate` function takes the analysis one step further: it generates a complete, self-contained Python script based on the dependency report. This is perfect for sharing a piece of your work or creating a reproducible example.

In [10]:
# Isolate the `process_data` function
result = isolate(process_data)

print(result.source)

import numpy as np
import textwrap

def process_data(data, name):
    """
    This function uses:
    1. An external library (numpy).
    2. A standard library (textwrap).
    3. A local function (get_greeting).
    """
    greeting = get_greeting(name)
    print(greeting)
    mean_value = np.mean(data)
    print(f'Mean of data: {mean_value}')
    long_message = f'The calculated mean was {mean_value}. This is a long message that needs to be wrapped nicely for display.'
    wrapped_message = textwrap.fill(long_message, width=40)
    print('\n--- Formatted Message ---')
    print(wrapped_message)
    return mean_value

def get_greeting(name: str) -> str:
    """A simple helper function."""
    return f'Hello, {name}!'



### Examining the Isolated Code

The generated script has a few distinct parts:

1.  **Imports**: All necessary `import` statements are placed at the top.

2.  **Variables**: Any required module-level variables come next (if present).

3.  **Definitions**: All required functions and classes (`get_greeting` and `process_data`) are included in the correct order.

The `isolate` helper returns a result object with:

- `source`: the generated Python source string
- `warnings`: unresolved name messages (also printed to stderr)
- `reqs_pypi` / `reqs_unknown`: inferred non-stdlib package names


### Running the Isolated Code

We can even execute the generated code directly to prove that it works.

In [11]:
# We can add a call to our function at the end of the script
runnable_script = result.source + "\n# --- Let's run it! ---\nprocess_data([10, 20, 30, 40], name='World')"

print("--- Executing Generated Script ---")
exec(runnable_script)

--- Executing Generated Script ---
Hello, World!
Mean of data: 25.0

--- Formatted Message ---
The calculated mean was 25.0. This is a
long message that needs to be wrapped
nicely for display.


## 5. Advanced: Isolating Multiple Entries

`deph` can also handle multiple entry points at once, intelligently merging their dependencies into a single script.

In [12]:
# A second, independent function
def say_goodbye():
    print("Goodbye!")

# Isolate both `process_data` and `say_goodbye` together
result2 = isolate(process_data, say_goodbye)

print(result2.source)


import numpy as np
import textwrap

def process_data(data, name):
    """
    This function uses:
    1. An external library (numpy).
    2. A standard library (textwrap).
    3. A local function (get_greeting).
    """
    greeting = get_greeting(name)
    print(greeting)
    mean_value = np.mean(data)
    print(f'Mean of data: {mean_value}')
    long_message = f'The calculated mean was {mean_value}. This is a long message that needs to be wrapped nicely for display.'
    wrapped_message = textwrap.fill(long_message, width=40)
    print('\n--- Formatted Message ---')
    print(wrapped_message)
    return mean_value

def get_greeting(name: str) -> str:
    """A simple helper function."""
    return f'Hello, {name}!'

def say_goodbye():
    print('Goodbye!')



As you can see, `deph` combined the dependencies for both functions, including `numpy`, `textwrap`, `get_greeting`, `process_data`, and `say_goodbye` in the final script.